In [120]:
import pandas as pd
import random
from PIL import Image, ImageDraw #Подключим необходимые библиотеки. 
import pydub
import os
import glob
from tqdm import tqdm
import librosa
from matplotlib import pyplot as plt
import librosa.display

In [128]:
def mp3_to_spectrogram(path,path_to_save,data):
    for file in tqdm(range(data.shape[0])) :
        files=path + data['path'][file]
        x , sr = librosa.load(files) # x - samples, sr - sample rate
        temp = os.path.splitext(files)[0]
        file_name=os.path.basename(temp)
    
        X = librosa.stft(x) # Short-time Fourier transform
        Xdb = librosa.amplitude_to_db(abs(X)) #Convert an amplitude spectrogram to dB-scaled spectrogram.
        plt.figure(figsize=(14, 5))
        librosa.display.specshow(Xdb, sr=sr,bins_per_octave=129,hop_length=50 ) #Display a spectrogram,Number of bins per octave = 129, Hann Window default  
        plt.axis('off')
        
        path_spect=path_to_save + file_name +'.png'
        plt.savefig(path_spect ,bbox_inches = 'tight',pad_inches = 0, dpi=100) #save spectrogram
        
    
        image = Image.open(path_spect) #open spectrogram 
        draw = ImageDraw.Draw(image) #create a drawing tool
        width = image.size[0] #Determine the width
        height = image.size[1] #Determine the height
        pix = image.load() #Upload pixel values
    
        for i in range(width):
            for j in range(height):
                a = pix[i, j][0]
                b = pix[i, j][1]
                c = pix[i, j][2]
                S = (a + b + c) // 3
                draw.point((i, j), (S, S, S))
        image.save(path_spect)
        plt.clf()

In [229]:
#way to train.tsv for all languages
path_ru=('D:/Voice/Rus/train.tsv')
path_en=('D:/Voice/en/train.tsv')
path_de=('D:/Voice/de/train.tsv')
patn_cn=('D:/Voice/CN/train.tsv')

In [230]:
df_ru = pd.read_csv(path_ru,sep='\t')
df_cn = pd.read_csv(patn_cn,sep='\t')
df_en = pd.read_csv(path_en,sep='\t')
df_de = pd.read_csv(path_de,sep='\t') 

In [231]:
#delete unnecessary columns
df_ru=df_ru.drop(['client_id', 'up_votes', 'down_votes','accent','sentence'], axis=1)
df_cn=df_cn.drop(['client_id', 'up_votes', 'down_votes','accent','sentence'], axis=1)
df_en=df_en.drop(['client_id', 'up_votes', 'down_votes','accent','sentence'], axis=1)
df_de=df_de.drop(['client_id', 'up_votes', 'down_votes','accent','sentence'], axis=1)

In [232]:
# delete all the values where there is NaN
df_ru = df_ru.dropna()
df_cn = df_cn.dropna()
df_en = df_en.dropna()
df_de = df_de.dropna()


In [233]:
#update indexes after dropna
df_ru.reset_index(drop=True, inplace=True)
df_cn.reset_index(drop=True, inplace=True)
df_en.reset_index(drop=True, inplace=True)
df_de.reset_index(drop=True, inplace=True)

In [234]:
df_ru_ru = df_ru[0:1499] # take the first 1500 elements
df_cn_cn = df_cn[0:1499]
df_en_en = df_en[0:1499]
df_de_de = df_de[0:1499]

In [235]:
df_ru_ru['language']='ru' #add a column with a language class (ru, en, de, cn)
df_cn_cn['language']='cn'
df_en_en['language']='en'
df_de_de['language']='de'

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [236]:
df_all = pd.concat([df_ru_ru, df_cn_cn, df_en_en, df_de_de], ignore_index=True) #concatenate 4 df into one df_all

In [237]:
df_all['gender']=df_all['gender'].apply(lambda x: 1 if x == 'male' else 0)# replace male = 1 female = 0

In [239]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5996 entries, 0 to 5995
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   path      5996 non-null   object
 1   age       5996 non-null   object
 2   gender    5996 non-null   int64 
 3   language  5996 non-null   object
dtypes: int64(1), object(3)
memory usage: 187.5+ KB


In [240]:
#path to records all languages
path_voice_ru=('D:/Voice/Rus/clips/')
path_voice_en=('D:/Voice/en/clips/')
path_voice_de=('D:/Voice/de/clips/')
path_voice_cn=('D:/Voice/CN/clips/')

In [241]:
path_to_save='D:/Voice/spect/'

In [ ]:
#start the spectrogram generation function

In [263]:
mp3_to_spectrogram(path_voice_ru,path_to_save,df_ru_ru)

In [264]:
mp3_to_spectrogram(path_voice_en,path_to_save,df_en_en)

In [265]:
mp3_to_spectrogram(path_voice_de,path_to_save,df_de_de)

In [266]:
mp3_to_spectrogram(path_voice_cn,path_to_save,df_cn_cn)

In [260]:
# replace the extension from .mp3 to .png in the file name in df_all - combined data frame
for i in range(df_all.shape[0]):
    df_all['path'][i]= df_all['path'][i][:-4] +'.png'

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [262]:
df_all.to_csv('D:\Voice\df_all.csv ' , index=False) # save df_all 